In [162]:
import requests
import pandas as pd
import numpy as np
import pytz
import time
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import LSTM, Dense

def get_stock_data(SYMB,Interval,Range):

    url = f'https://query1.finance.yahoo.com/v8/finance/chart/{SYMB}?region=US&lang=en-US&includePrePost=false&interval={Interval}&useYfid=true&range={Range}&corsDomain=finance.yahoo.com&.tsrc=finance'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: Request failed with status code {response.status_code}")

    # Extract data from the JSON
    meta_data = data['chart']['result'][0]['meta']
    timestamp = data['chart']['result'][0]['timestamp']
    quote_data = data['chart']['result'][0]['indicators']['quote'][0]

    # Convert timestamp to Indian Standard Time (IST)
    utc = pytz.timezone('UTC')
    ist = pytz.timezone('Asia/Kolkata')
    ist_time = [datetime.fromtimestamp(ts, tz=utc).astimezone(ist) for ts in timestamp]

    # Create a DataFrame from the data
    df = pd.DataFrame({
        'symbol': [meta_data['symbol']] * len(timestamp),
        'Date': ist_time,
        'Open': quote_data['open'],
        'High': quote_data['high'],
        'Low': quote_data['low'],
        'Close': quote_data['close']
    })
    df.dropna(inplace=True)
    return df

def next_candle(SYMB, Interval, Range, time_steps):
    
    start_time = time.time()
    
    df = get_stock_data(SYMB,Interval,Range)
    data = df[['Open', 'High', 'Low', 'Close']].values
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_normalized = scaler.fit_transform(data)
    time_steps = time_steps

    X, y = [], []
    for i in range(len(data_normalized) - time_steps):
        X.append(data_normalized[i:i+time_steps])
        y.append(data_normalized[i+time_steps])

    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, 4)))
    model.add(LSTM(50))
    model.add(Dense(4))  # 4 output nodes for predicting 'Open', 'High', 'Low', 'Close'

    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

    y_pred = model.predict(X_test)

    # Accuracy
    y_pred_original = scaler.inverse_transform(y_pred)
    y_test_original = scaler.inverse_transform(y_test)

    def mape(y_true, y_pred):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    accuracy = 100 - mape(y_test_original, y_pred_original)
#     print("Accuracy: {:.2f}%".format(accuracy))

    # Prediction
    last_time_step_data = scaled_data[-time_steps:]
    next_future_value = model.predict(np.array([last_time_step_data]))
    next_future_value = scaler.inverse_transform(next_future_value)
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    print('\n')
    print('\n')
    print(f"Next Future {Interval} Value Predictions:")
    print('----------------------------------')
    print(f"Open: {next_future_value[0][0]}")
    print(f"High: {next_future_value[0][1]}")
    print(f"Low: {next_future_value[0][2]}")
    print(f"Close: {next_future_value[0][3]}")
    print('\n')
    print("Model Accuracy: {:.2f}%".format(accuracy))
    print("Prediction Time: {:.6f} seconds".format(execution_time))

next_candle('RELIANCE.NS', '1m', '1d', 20) # You can select intervals like 1m 5m or 15m and 1day -- 1d 7d 30d data

Epoch 1/50
9/9 [==============================] - 5s 23ms/step - loss: 0.0668
Epoch 2/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0152
Epoch 3/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0107
Epoch 4/50
9/9 [==============================] - 0s 22ms/step - loss: 0.0063
Epoch 5/50
9/9 [==============================] - 0s 22ms/step - loss: 0.0054
Epoch 6/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0046
Epoch 7/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0041
Epoch 8/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0036
Epoch 9/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0032
Epoch 10/50
9/9 [==============================] - 0s 25ms/step - loss: 0.0030
Epoch 11/50
9/9 [==============================] - 0s 24ms/step - loss: 0.0029
Epoch 12/50
9/9 [==============================] - 0s 23ms/step - loss: 0.0029
Epoch 13/50
9/9 [==============================] - 0s 23ms/st